In [1]:
using Revise
@time using FUSE
using Plots;
FUSE.logging(Logging.Info; actors=Logging.Info);

[ Info: Precompiling Revise [295af30f-e4ad-537b-8983-00126c2a3abe]
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Revise [295af30f-e4ad-537b-8983-00126c2a3abe]
│   exception = Required dependency JuliaInterpreter [aa1ae85d-cabe-5617-a682-6adf51b2e16a] failed to load from a cache file.
└ @ Base loading.jl:1818
[ Info: Precompiling JuliaInterpreter [aa1ae85d-cabe-5617-a682-6adf51b2e16a]
[ Info: Precompiling LoweredCodeUtils [6f1432cf-f94c-5a45-995e-cdbf5db27b0b]


 20.905548 seconds (36.99 M allocations: 2.459 GiB, 4.66% gc time, 5.56% compilation time: 18% of which was recompilation)


In [2]:
save_folder = "/Users/slendebroek/FUSE_memory" # change to your save_folder

if !isdir(save_folder)
    mkdir(save_folder)
end

In [20]:
old_dir = pwd()
mkpath(save_folder)
try
    cd(save_folder) # this is to save temporary distributed files in working folder
    FUSE.parallel_environment("localhost",5)
finally
    cd(old_dir)
end
display(pwd())


Working with 4 processes on F-F9Q72P69Y


"/Users/slendebroek/.julia/dev/FUSE/examples/actors"

In [21]:
#minimal working example for loading and saving dd using pmap
using Distributed
@everywhere import IMAS
@everywhere import FUSE

@everywhere function load_case(input_folder)
    IMAS.json2imas(joinpath(input_folder,"dd.json")),FUSE.json2act(joinpath(input_folder,"act.json")) 
end
   

@everywhere function run_case_try(folder,input_folder)
    if !isdir(folder)
        mkdir(folder)
    end
    
    try
        dd,act = load_case(input_folder)
        
        FUSE.ActorNoOperation(dd,act)
    
        FUSE.save(FUSE.memtrace, joinpath(folder, "memtrace.txt"))
        IMAS.imas2json(dd,joinpath(folder,"outputdd.json"))
    catch e
        println(e)
    end
    return nothing
end

@everywhere function run_case_no_try(folder,input_folder)

    if !isdir(folder)
        mkdir(folder)
    end
    
    dd,act = load_case(input_folder)

    FUSE.ActorNoOperation(dd,act)

    FUSE.save(FUSE.memtrace, joinpath(folder, "memtrace.txt"))
    IMAS.imas2json(dd,joinpath(folder,"outputdd.json"))

    return nothing
end


@everywhere function _run_case_no_try(folder,input_folder)
    tmp = run_case_no_try(folder,input_folder)
    tmp = nothing
    Gc.gc()
end


In [26]:
N_runs = 5000
to_run = [joinpath(save_folder,"$i") for i in 1000:1:1000 + N_runs]
input_folder = save_folder

run_case_no_try(to_run[1],input_folder); # test 1 case

[ Info: NoOperation


In [ ]:
pmap(folder -> run_case_no_try(folder, input_folder), to_run)    # load & save without gc
#pmap(folder -> _run_case_no_try(folder, input_folder), to_run)  # warpped function with gc

In [ ]:
# to see memory output

f1 = readdir(save_folder)[end-3]
@show f1
mem = FUSE.load(FUSE.MemTrace(),joinpath(save_folder,f1,"memtrace.txt"))

plot(mem,0)

In [ ]:
# MWE using the optimizer (setup part)

ini,act = FUSE.case_parameters(:KDEMO);


### Ini settings

ini.equilibrium.B0 = 6.5

ini.nbi.beam_energy = 5e6# ↔ [1e5,1e7]
ini.nbi.power_launched = 10e6
ini.requirements.log10_flattop_duration = log10(7200.0)#log10(1800.0) # log10(7200.0)
ini.requirements.power_electric_net = 250e6
ini.equilibrium.κ = 0.95
ini.equilibrium.ip = 15e6
ini.equilibrium.pressure_core = 1e4

ini.tf.technology = :HTS
ini.oh.technology = :HTS


### Act settings
act.ActorEquilibrium.model = :TEQUILA
act.ActorStabilityLimits.models = [:safety_factor_rho_08_lt_2]
act.ActorCoreTransport.model = :FluxMatcher

act.ActorWholeFacility.update_plasma = true

act.ActorPFcoilsOpt.optimization_scheme = :none
act.ActorPedestal.only_powerlaw = true

act.ActorTGLF.user_specified_model = "sat1_em_iter"
act.ActorFluxMatcher.rho_transport = 0.2:0.05:0.85
act.ActorFluxMatcher.evolve_densities = :flux_match
act.ActorFluxMatcher.max_iterations = 1000

ini.ec_launchers.power_launched = ini.ec_launchers.power_launched[1] ↔ [1e6, 200e6];
ini.core_profiles.zeff = ini.core_profiles.zeff ↔ [1.1, 2.8]
ini.core_profiles.greenwald_fraction = ini.core_profiles.greenwald_fraction ↔ [0.8, 1.3]

ini.nbi.beam_energy = 1e6 ↔ [1e5,1e7]
ini.nbi.power_launched = 10e6 ↔ [1e5,40e6]

ini.equilibrium.δ = ini.equilibrium.δ ↔ [0.3,0.7]
#ini.equilibrium.ζ = ini.equilibrium.ζ ↔ [0,0.2]
ini.equilibrium.κ = 0.95 # k set to be 95% of maximum controllable value
ini.equilibrium.B0 = ini.equilibrium.B0 ↔ [3.0, 12.0]
ini.equilibrium.ip = ini.equilibrium.ip ↔ [5.0e6, 22e6]
ini.equilibrium.R0 = ini.equilibrium.R0 ↔ [5.0, 10.0];
ini.requirements.log10_flattop_duration = log10(7200.0) ↔ [log10(3600.0), log10(1000.0 * 3600.0)];
ini.requirements.power_electric_net = 250e6

ini.tf.technology = :HTS ↔ (:HTS, :LTS);
ini.oh.technology = :HTS ↔ (:HTS, :LTS);

###

dd_init = FUSE.init(ini,act);
display(plot(dd_init.equilibrium))
display(plot(dd_init.core_sources))
display(plot(dd_init.core_profiles))

# FUSE comes with a library of objective and constraints functions
OFL = deepcopy(FUSE.ObjectiveFunctionsLibrary)
CFL = deepcopy(FUSE.ConstraintFunctionsLibrary)
println("== OBJECTIVE FUNCTIONS ==")
display(OFL)
println()
println("== CONSTRAINT FUNCTIONS ==")
display(CFL)


objective_functions = [OFL[:min_βn], OFL[:min_capital_cost]]#, OFL[:max_log10_flattop]]
constraint_functions = [CFL[:min_required_power_electric_net]]
println("== OBJECTIVE FUNCTIONS ==")
display(objective_functions)
println()
println("== CONSTRAINT FUNCTIONS ==")
display(constraint_functions)


In [ ]:
# run the optimization

save_folder_opt = joinpath(save_folder,"opt")
if !isdir(save_folder_opt)
    mkdir(save_folder_opt)
end

optimization_parameters = Dict(
    :N => 4, # even number (population size)
    :iterations => 5,   # number of genetic algo iterations
    :continue_state => missing,
    :save_folder => save_folder_opt)

# run optimization

# This one runs no real physics just initializes
state = FUSE.workflow_multiobjective_optimization(ini, act, FUSE.ActorNoOperation, objective_functions, constraint_functions; optimization_parameters...);

# This one runs with physics
#state = FUSE.workflow_multiobjective_optimization(ini, act, FUSE.ActorWholeFacility, objective_functions, constraint_functions; optimization_parameters...);
